In [1]:
from datasets import Dataset, load_dataset
from dotenv import load_dotenv
from sentence_transformers import CrossEncoder
# from ..model.llm import LLM
from pandas import DataFrame, read_parquet
import numpy as np
from sentence_transformers import CrossEncoder

load_dotenv("../.env")

True

In [ ]:
model_name = None
model_size = "7"
adapter = False
k_retriever = 10
k_reranker = 10
reranker_name = "../models/itrf-"
device_reranker = "cuda:1"
device_embedding = "cuda:1"
device_llm = "cuda:1"



In [ ]:
# Search (same for all models)


In [ ]:
# Reranker
reranker = None

if reranker_name:
    reranker = CrossEncoder(reranker_name, num_labels=1, device=device_reranker)

def rerank(retrieved_contexts, question, k_reranker=10):
    scores = reranker.predict([(question, context) for context in retrieved_contexts])
    sorted_contexts = [context for _, context in sorted(zip(scores, retrieved_contexts), reverse=True)]
    return sorted_contexts[:k_reranker]

In [ ]:
llm = LLM(size=model_size, quantized=False, model_path=model_name, adapter=adapter)

# Set

In [3]:
from datasets import load_dataset, Dataset
from langchain.embeddings import HuggingFaceBgeEmbeddings
from qdrant_client import models, QdrantClient
from langchain.vectorstores import Qdrant
import time
from datetime import timedelta
import pandas as pd
from pandas import DataFrame
import argparse

In [4]:
d = load_dataset("natural_questions", split="validation", streaming=True)
d = iter(d)
dp = next(d)
for i in range(10):
    dp = next(d)

id = dp["id"]
qu = dp["question"]["text"]

ans = None
for sa in dp["annotations"]["short_answers"]:
    if len(sa["text"]) > 0:
        ans = sa["text"][0]
        break

qu, ans

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

('which mode is used for short wave broadcast service', 'MFSK')

In [6]:
# d = load_dataset("mandarjoshi/trivia_qa", 'rc.wikipedia.nocontext', split="validation", streaming=True)
d = load_dataset("mandarjoshi/trivia_qa", 'rc.web.nocontext', split="validation", streaming=True)
d = iter(d)
dp = next(d)

id = dp["question_id"]
qu = dp["question"]
ans = dp["answer"]["value"]

qu, ans

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

('Who was the man behind The Chipmunks?', 'David Seville')

In [8]:
d = load_dataset("hotpot_qa", "distractor", split="validation", streaming=True)

d = iter(d)
dp = next(d)

id = dp["id"]
qu = dp["question"]
ans = dp["answer"]

qu, ans

('Were Scott Derrickson and Ed Wood of the same nationality?', 'yes')

In [ ]:
for sample in pd.read_parquet("data/parquet/mandarjoshi_trivia_qa_rc.wikipedia.nocontext_validation.parquet"):
    contexts = sample["contexts"]
    print(contexts["text"])

# Retrieval

In [2]:
itrf = read_parquet("../data/dataset/itrf_evaluation_rererank_base.parquet")
len(itrf)

31742

In [3]:
# Remove duplicates
itrf.drop_duplicates(subset=["query", "ground_truth", "src", "id"], inplace=True)

# Remove examples with na fields
itrf.dropna(inplace=True)

In [4]:
itrf.index = range(len(itrf))

In [5]:
len(itrf)

31742

In [4]:
contexts = itrf.iloc[0]["contexts"]
len(contexts[:])

100

In [11]:
sort_ind = np.argsort([c["reranker_softmax"] for c in contexts])
contexts = contexts[sort_ind]

# Query
print(itrf.iloc[0]["query"])
contexts[-1]

For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I took $1 million from friends Scenario 2 | I submitted my timecard at work with precisely the correct hours I worked.
Options: 1) Wrong, Wrong, 2) Wrong, Not wrong, 3) Not wrong, Wrong, 4) Not wrong, Not wrong



{'id': '12938894',
 'original_context': False,
 'reranker_score': 0.976978600025177,
 'reranker_softmax': 0.2598695158958435,
 'retriever_score': 0.63741565,
 'retriever_softmax': 0.010054686052146806,
 'src': 'oscar',
 'text': 'the interaction occurs between friends, and the act is rated as more morally wrong overall. A more formal test of these competing hypotheses appears to await data. I’ll be sure to get right on that personally; just as soon as I’m done being addicted to this game in the next three or four years.'}

In [10]:
[c["reranker_softmax"] for c in contexts]

[7.47909361962229e-05,
 7.740228465991095e-05,
 7.740228465991095e-05,
 0.000156113164848648,
 0.0001740203151712194,
 0.0002049776812782511,
 0.00027580803725868464,
 0.00027779542142525315,
 0.00028498496976681054,
 0.0003416207619011402,
 0.00038805115036666393,
 0.00039261806523427367,
 0.0003971851256210357,
 0.0004142938123550266,
 0.00042401475366204977,
 0.0004267686454113573,
 0.0004317693237680942,
 0.0004930343711748719,
 0.000493193743750453,
 0.0004941489896737039,
 0.0005029336316511035,
 0.000537477491889149,
 0.0005676859291270375,
 0.0005801974912174046,
 0.0006103533669374883,
 0.0006612141150981188,
 0.0007241718121804297,
 0.0007326455670408905,
 0.0007591660832986236,
 0.0008086252491921186,
 0.0009279329678975046,
 0.0009299360681325197,
 0.0009450815268792212,
 0.0009534443379379809,
 0.0009692144813016057,
 0.000997181050479412,
 0.0010045142844319344,
 0.0010098546044901013,
 0.001026306301355362,
 0.0010454598814249039,
 0.001098298467695713,
 0.00111428333912

In [10]:
sort_ind = np.argsort([c["id"] for c in contexts])
contexts[sort_ind]

array([{'id': '10016334', 'original_context': False, 'retriever_score': 0.62567085, 'retriever_softmax': 0.009869421889358751, 'src': 'oscar', 'text': 'If it is a) then how does it not contradict what you initially said (this is almost verbatim what you initially said, just with a “do mean” instead of a “don’t mean”) and if it is b) what is your non-random assessment of whether you’ll adopt the “someone else’s” vantage point on morality instead of some totally different other persons vantage point on moral matters?\nWho says that vetting necessarily involves testing to see if a moral code fragment would constitute an improvement as judged from my current moral code? I never said that. I was responding to your worry:'},
       {'id': '10029618', 'original_context': False, 'retriever_score': 0.6372263, 'retriever_softmax': 0.010051699218039464, 'src': 'oscar', 'text': 'here at the bank, I change the another ( ), another worker. I: Oh, yeah. 12B: I heard that, I heard her taboo. It is not

In [11]:
for c in contexts[:-2]:
    if c["id"] == "Personal narrative":
        print(c["text"])

to persuade the listener that they would have acted similarly; the speaker extends their moral stance to the listener as well. The notion that "this happened to me" is the justification of storytelling rights for all personal narrative, defence of one's actions is an integral part of this moral negotiation. More than any other topic of personal narrative, one talks more giving evidence of fairness or unfairness, drawing sympathy, approval, exoneration, understanding, or amusement from their audience. Even some surface-level badmouthing of self can function as a reassertion of one's merit or value. The self-depreciator uses ventrilloquation (using one's voice to an enacted another) to act out or distance


In [2]:
base_model = "../models/itrf_reranker-large/final_model"
ce = CrossEncoder(base_model, num_labels=1)

In [ ]:
itrf.apply(lambda x: ce.predict([x["query"], x["src"]]), axis=1)

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
base_model = "meta-llama/Llama-2-7b-chat-hf"
model = LlamaForCausalLM.from_pretrained(
                    base_model,
                    device_map="cuda:0",
                    attn_implementation="flash_attention_2",
                    torch_dtype="auto", # torch.bfloat16,  # you may change it with different models
                )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [3]:
model = model.eval()

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.add_special_tokens({"additional_special_tokens": ["<>", "<inst_e>"]})
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"

# Generation

In [97]:
from pandas import read_parquet

itrf = read_parquet("../data/dataset/itrf_evaluation_retrieval_processed.parquet")
itrf

,split,query,ground_truth,contexts,src,id,task,domain,query_plain
0,validation,In the discharge checklist recommended by the ...,"2) The nurse, patient, and ward clerk.","[{'id': '9373665', 'llm_score': 0.998656597973...",cais/mmlu,f5dab0cd07f621d72dca70cfe10b4284,mc,trivia,In the discharge checklist recommended by the ...
1,validation,What is the 'security dilemma' that faces weak...,1) The inability of the state to provide stabi...,"[{'id': 'National security', 'llm_score': 0.99...",cais/mmlu,1f037ba8922e166ccc495fabd299947a,mc,trivia,What is the 'security dilemma' that faces weak...
2,validation,A complaint filed on behalf of a woman against...,2) One of the two defendants probably caused t...,"[{'id': '2251869', 'llm_score': 0.992136416490...",cais/mmlu,4046ac8b8013af4810e57c2100024ec3,mc,trivia,A complaint filed on behalf of a woman against...
3,validation,For which of these two scenarios does the main...,"1) Wrong, Wrong","[{'id': '3820166', 'llm_score': 0.987903482771...",cais/mmlu,cbec365641199984a2ebb122d5f33d0f,mc,trivia,For which of these two scenarios does the main...
4,validation,What is believed to be the reason for the deve...,2) the construction of canals for irrigation,"[{'id': 'The Roads of Chaco', 'llm_score': 0.9...",cais/mmlu,346032614a029da52cb8c34dd1a4765d,mc,trivia,What is believed to be the reason for the deve...
...,...,...,...,...,...,...,...,...,...
1595,validation,"Who won more academy awards, Joseph L. Mankie...",Joseph Leo Mankiewicz,"[{'id': '5134151', 'llm_score': 0.979858651346...",hotpot_qa,219269a844cc7a4cf12badc42ab6c1a0,mc,trivia,"Who won more academy awards, Joseph L. Mankie..."
1596,validation,Legends of the Fall was based on a novella by...,James Harrison,"[{'id': 'Speculative poetry', 'llm_score': 0.9...",hotpot_qa,e5d2c96ecf21345ee12e38ad6eb29b52,mc,trivia,Legends of the Fall was based on a novella by...
1597,validation,Which group of people in West Africa with sign...,The Yoruba,"[{'id': '6258015', 'llm_score': 0.982515325570...",hotpot_qa,38fe3455e675cdb77a4c14c6151e03d1,mc,trivia,Which group of people in West Africa with sign...
1598,validation,What team did Kenny Brown manage following the...,Chelmsford,"[{'id': '12431963', 'llm_score': 0.92583652652...",hotpot_qa,8c2cafecca8a96923046fad4ccf50695,mc,trivia,What team did Kenny Brown manage following the...


In [62]:
itrf.iloc[0]["contexts"][0]

{'id': '4801225',
 'original_context': False,
 'retriever_score': 0.6776359,
 'retriever_softmax': 0.010689126054818308,
 'src': 'oscar',
 'text': 'Mjd\'s answer is correct with respect to where to use "I" or "me" but there is more to the question. In what order do you put them? Which of these sentences would you say?\nIt is normal to put yourself after other people. This is a matter of courtesy but would you extend that courtesy to a dog? I wouldn\'t, I\'d rather go with the b) sentences.\nWhat if it weren\'t a dog but an inanimate object like a computer? Which of these sentences would you use?'}

In [58]:
import numpy as np
score = np.argmax([c["llm_score"] for c in itrf.iloc[0]["contexts"]])
itrf.iloc[0]["contexts"][score]

{'id': '1050385',
 'llm_score': 0.9922882308426534,
 'llm_softmax': 1.0,
 'llm_true_score': 3.2618087066963705e-17,
 'llm_true_softmax': 3.7562018468836085e-06,
 'original_context': False,
 'predicted': '\n\n2) Wrong, Not wrong',
 'reranker_score': 0.8862756490707397,
 'reranker_softmax': 0.23574316501617432,
 'retriever_score': 0.6349007,
 'retriever_softmax': 0.010015014869478407,
 'src': 'oscar',
 'text': '“me” or “my”.'}

In [63]:
itrf.iloc[0]["ground_truth"]

'2) Wrong, Not wrongWrong, Not wrong'

We noticed that the answers were retrieved incorrectly so we reloaded them

In [64]:
itrf["query_plain"] = itrf["query"].apply(lambda x: x.split("\nOptions:")[0])

In [35]:
from datasets import load_dataset
d = load_dataset("cais/mmlu", "all", split="test")
shuffled = iter(d.shuffle(seed=4048))

total = 0
skipped = 0
processed = 0
not_matching = 0

for i in range(7500):
    dp = next(shuffled)
    ex = itrf.iloc[i]

    if ex["src"] != "cais/mmlu":
        print("Different source", ex["src"], "cais/mmlu")
        break

    if ex["query_plain"] != dp["question"]:
        total += 1
        while ex["query_plain"] != dp["question"]:
            print("Skipping question", ex["query"], dp["question"])
            dp = next(shuffled)
            skipped = skipped + 1
            if skipped > 7500:
                break
        print("Found question", ex["query"], dp["question"])
    if ex["query_plain"] == dp["question"]:
        processed += 1
        itrf.loc[i, "ground_truth"] = str(dp["answer"]+1) + ") " + str(dp["choices"][dp["answer"]])
    if int(ex["ground_truth"][0])-1 != dp["answer"]:
         not_matching += 1
         print(f"Different context", int(ex["ground_truth"][0])-1, dp["answer"])

Skipping question How many electrons does gold (Au) possess that have a quantum number l = 2?
Options: 1) 9, 2) 23, 3) 29, 4) 79
 An artist doing a pencil drawing could use which of the following techniques to add depth to her or his drawing?
Found question How many electrons does gold (Au) possess that have a quantum number l = 2?
Options: 1) 9, 2) 23, 3) 29, 4) 79
 How many electrons does gold (Au) possess that have a quantum number l = 2?
Skipping question Noncompetitive inhibition differs from uncompetitive inhibition in that a noncompetitive inhibitor binds to an allosteric site on the enzyme and prevents it from catalyzing a reaction, whereas uncompetitive inhibitors bind to the enzymesubstrate complex and prevent catalysis. Increasing the substrate concentration would have which of the following effects?
Options: 1) Increasing impact of uncompetitive inhibitor and decreasing concentration of noncompetitive inhibitor, 2) Decreasing impact of uncompetitive inhibitor and increasing

In [66]:
len(itrf[itrf["src"]== "cais/mmlu"])

7471

In [67]:
processed

7471

In [68]:
itrf.to_parquet("../data/dataset/.parquet")

# Eval

In [1]:
from pandas import read_parquet

itrf = read_parquet("../data/dataset/itrf_evaluation_generation_llmware_llmware_mse.parquet")


In [2]:
itrf

,split,query,ground_truth,contexts,src,id,task,domain,query_plain
0,validation,In the discharge checklist recommended by the ...,"2) The nurse, patient, and ward clerk.","[{'id': '3759086', 'llm_score': 0.990682598318...",cais/mmlu,f5dab0cd07f621d72dca70cfe10b4284,mc,trivia,In the discharge checklist recommended by the ...
1,validation,What is the 'security dilemma' that faces weak...,1) The inability of the state to provide stabi...,"[{'id': 'Balancing (international relations)',...",cais/mmlu,1f037ba8922e166ccc495fabd299947a,mc,trivia,What is the 'security dilemma' that faces weak...
2,validation,A complaint filed on behalf of a woman against...,2) One of the two defendants probably caused t...,"[{'id': '9875000', 'llm_score': 0.976631289535...",cais/mmlu,4046ac8b8013af4810e57c2100024ec3,mc,trivia,A complaint filed on behalf of a woman against...
3,validation,For which of these two scenarios does the main...,"1) Wrong, Wrong","[{'id': '3820166', 'llm_score': 0.973908787801...",cais/mmlu,cbec365641199984a2ebb122d5f33d0f,mc,trivia,For which of these two scenarios does the main...
4,validation,What is believed to be the reason for the deve...,2) the construction of canals for irrigation,"[{'id': 'Las Haldas', 'llm_score': 0.948151379...",cais/mmlu,346032614a029da52cb8c34dd1a4765d,mc,trivia,What is believed to be the reason for the deve...
...,...,...,...,...,...,...,...,...,...
1595,validation,"Who won more academy awards, Joseph L. Mankie...",Joseph Leo Mankiewicz,"[{'id': '13564022', 'llm_score': 0.96435913413...",hotpot_qa,219269a844cc7a4cf12badc42ab6c1a0,mc,trivia,"Who won more academy awards, Joseph L. Mankie..."
1596,validation,Legends of the Fall was based on a novella by...,James Harrison,"[{'id': '11701774', 'llm_score': 0.91306868172...",hotpot_qa,e5d2c96ecf21345ee12e38ad6eb29b52,mc,trivia,Legends of the Fall was based on a novella by...
1597,validation,Which group of people in West Africa with sign...,The Yoruba,"[{'id': 'Women in Africa', 'llm_score': 0.9494...",hotpot_qa,38fe3455e675cdb77a4c14c6151e03d1,mc,trivia,Which group of people in West Africa with sign...
1598,validation,What team did Kenny Brown manage following the...,Chelmsford,"[{'id': '12431963', 'llm_score': 0.87640653970...",hotpot_qa,8c2cafecca8a96923046fad4ccf50695,mc,trivia,What team did Kenny Brown manage following the...


In [104]:
for i in itrf["src"].unique():
    df = itrf[itrf["src"] == i].sample(400, random_state=4048)
    print(i, len(df))

cais/mmlu 400
natural_questions 400
mandarjoshi/trivia_qa 400
hotpot_qa 400


In [3]:
datasets = itrf["src"].unique()

def rank_context(sample, get_answer=False):
    # Select answer by reranker times llm score
    # Get context with the highest score = llm_softmax * reranker_softmax
    contexts = sample["all_contexts"]
    highest_score_context = max(contexts, key=lambda x: x['reranker_softmax'] * x['llm_softmax'])

    if get_answer:
        return highest_score_context["predicted"].strip()
    else:
        return [highest_score_context["text"]]

# Prepare data
itrf["question"] = itrf["query"]
itrf.rename(columns={"contexts": "all_contexts",}, inplace=True)
itrf["answer"] = itrf.apply(rank_context, axis=1, get_answer=True)
itrf["ground_truths"] = itrf["ground_truth"].apply(lambda x: [x])  # "ground_truth": "ground_truths"
itrf["contexts"] = itrf.apply(rank_context, axis=1)

itrf

,split,query,ground_truth,all_contexts,src,id,task,domain,query_plain,question,answer,ground_truths,contexts
0,validation,In the discharge checklist recommended by the ...,"2) The nurse, patient, and ward clerk.","[{'id': '3759086', 'llm_score': 0.990682598318...",cais/mmlu,f5dab0cd07f621d72dca70cfe10b4284,mc,trivia,In the discharge checklist recommended by the ...,In the discharge checklist recommended by the ...,"The nurse, patient, and ward clerk.","[2) The nurse, patient, and ward clerk.]",[completing this document: You must complete t...
1,validation,What is the 'security dilemma' that faces weak...,1) The inability of the state to provide stabi...,"[{'id': 'Balancing (international relations)',...",cais/mmlu,1f037ba8922e166ccc495fabd299947a,mc,trivia,What is the 'security dilemma' that faces weak...,What is the 'security dilemma' that faces weak...,1) The inability of the state to provide stabi...,[1) The inability of the state to provide stab...,[Conceptualizing and understanding the Nationa...
2,validation,A complaint filed on behalf of a woman against...,2) One of the two defendants probably caused t...,"[{'id': '9875000', 'llm_score': 0.976631289535...",cais/mmlu,4046ac8b8013af4810e57c2100024ec3,mc,trivia,A complaint filed on behalf of a woman against...,A complaint filed on behalf of a woman against...,There are grounds for the fact-finder to infer...,[2) One of the two defendants probably caused ...,[Though Purches suffered two unique types of i...
3,validation,For which of these two scenarios does the main...,"1) Wrong, Wrong","[{'id': '3820166', 'llm_score': 0.973908787801...",cais/mmlu,cbec365641199984a2ebb122d5f33d0f,mc,trivia,For which of these two scenarios does the main...,For which of these two scenarios does the main...,Scenario 1.,"[1) Wrong, Wrong]",[I took out of the song that it’s being sung f...
4,validation,What is believed to be the reason for the deve...,2) the construction of canals for irrigation,"[{'id': 'Las Haldas', 'llm_score': 0.948151379...",cais/mmlu,346032614a029da52cb8c34dd1a4765d,mc,trivia,What is believed to be the reason for the deve...,What is believed to be the reason for the deve...,The construction of canals for irrigation.,[2) the construction of canals for irrigation],"[With respect to architecture, this civilizati..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,validation,"Who won more academy awards, Joseph L. Mankie...",Joseph Leo Mankiewicz,"[{'id': '13564022', 'llm_score': 0.96435913413...",hotpot_qa,219269a844cc7a4cf12badc42ab6c1a0,mc,trivia,"Who won more academy awards, Joseph L. Mankie...","Who won more academy awards, Joseph L. Mankie...",Joseph L. Mankiewicz.,[Joseph Leo Mankiewicz],[Mankiewicz is best known for his collaboratio...
1596,validation,Legends of the Fall was based on a novella by...,James Harrison,"[{'id': '11701774', 'llm_score': 0.91306868172...",hotpot_qa,e5d2c96ecf21345ee12e38ad6eb29b52,mc,trivia,Legends of the Fall was based on a novella by...,Legends of the Fall was based on a novella by...,Edgar Allan Poe,[James Harrison],[death already so here the demon could perhaps...
1597,validation,Which group of people in West Africa with sign...,The Yoruba,"[{'id': 'Women in Africa', 'llm_score': 0.9494...",hotpot_qa,38fe3455e675cdb77a4c14c6151e03d1,mc,trivia,Which group of people in West Africa with sign...,Which group of people in West Africa with sign...,Sierra Leoneans,[The Yoruba],"[Working with journalists, film crews, humanit..."
1598,validation,What team did Kenny Brown manage following the...,Chelmsford,"[{'id': '12431963', 'llm_score': 0.87640653970...",hotpot_qa,8c2cafecca8a96923046fad4ccf50695,mc,trivia,What team did Kenny Brown manage following the...,What team did Kenny Brown manage following the...,Not Found.,[Chelmsford],[No doubt to head off simmering fan discontent...


In [94]:
from dotenv import load_dotenv
from datasets import Dataset
load_dotenv("../.env")

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_similarity,
)
from ragas import evaluate

resultss = []

for d in datasets[:2]:
    df = itrf[itrf["src"] == d].sample(1, random_state=4048)
    ds = Dataset.from_pandas(df)
    results = evaluate(ds, metrics=[
        faithfulness, 
        answer_relevancy, 
        answer_similarity, 
        context_recall, 
        context_precision
    ])
    results["dataset"] = d
    resultss.append(results)

resultss

evaluating with [faithfulness]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


...

evaluating with [context_recall]


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


evaluating with [context_precision]


100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


ValueError: Unknown format code 'f' for object of type 'str'

In [96]:
results.to_pandas()

,split,query,ground_truth,all_contexts,src,id,task,domain,query_plain,question,answer,ground_truths,contexts,__index_level_0__,faithfulness,answer_relevancy,answer_similarity,context_recall,context_precision
0,validation,who's playing in the sec championship 2017,Georgia Bulldogs,"[{'id': '2018 SEC Championship Game', 'llm_sco...",natural_questions,4fe018121e1c97c7e5463e5027ccb9e4,mc,trivia,who's playing in the sec championship 2017,who's playing in the sec championship 2017,The SEC Championship Game in 201,[Georgia Bulldogs],[The SEC Championship Game is an annual Americ...,8177,1.0,0.828086,0.805068,1.0,0.0


In [95]:
import json 
json.dump(resultss, open("results.json", "w"))

In [45]:
itrf

,split,query,ground_truth,contexts,src,id,task,domain,query_plain,question
0,validation,For which of these two scenarios does the main...,"2) Wrong, Not wrong","[{'id': '14458516', 'llm_score': 0.0, 'llm_sof...",cais/mmlu,f990b1598e17cb38db40b0d1033b4c5a,mc,trivia,For which of these two scenarios does the main...,For which of these two scenarios does the main...
1,validation,"The term “insanity” is a:\nOptions: 1) legal, ...","1) legal, non-psychiatric term for certain typ...","[{'id': 'Insanity', 'llm_score': 1.0, 'llm_sof...",cais/mmlu,1ae9f5d44c79ff395f8fd1cb90d3de1b,mc,trivia,The term “insanity” is a:,"The term “insanity” is a:\nOptions: 1) legal, ..."
2,validation,"What is true for a type-Ia (""type one-a"") supe...",1) This type occurs in binary systems.,"[{'id': '4694198', 'llm_score': 1.0, 'llm_soft...",cais/mmlu,838fb96c08aa2f7d78fe0d2dcc0fcbbf,mc,trivia,"What is true for a type-Ia (""type one-a"") supe...","What is true for a type-Ia (""type one-a"") supe..."
3,validation,What is the name of the bodhisattva in Japan ...,3) Kannon,"[{'id': 'Glossary of Japanese Buddhism', 'llm_...",cais/mmlu,d084eec0de134771158ad0029f3c76e6,mc,trivia,What is the name of the bodhisattva in Japan ...,What is the name of the bodhisattva in Japan ...
4,validation,Which of the following is a strategy that the ...,4) Creating a school safety and response team,"[{'id': '7065282', 'llm_score': 0.999607104940...",cais/mmlu,adc520c8eabb1fffeb7a2bf0ed7f9e19,mc,trivia,Which of the following is a strategy that the ...,Which of the following is a strategy that the ...
...,...,...,...,...,...,...,...,...,...,...
31737,validation,At what height does State Route 160 start?,4145 ft,"[{'id': 'Virginia State Route 160', 'llm_score...",hotpot_qa,e4bdb4d584b5bc74373a423b13775aa3,mc,trivia,At what height does State Route 160 start?,At what height does State Route 160 start?
31738,validation,What team was Horace Grant's twin brother trad...,Portland Trail Blazers,"[{'id': 'Jerai Grant', 'llm_score': 0.0, 'llm_...",hotpot_qa,00a5b537346e5102db6a18ca327128d2,mc,trivia,What team was Horace Grant's twin brother trad...,What team was Horace Grant's twin brother trad...
31739,validation,What was the position of Jean Baptiste Androue...,supervisor of the royal works,"[{'id': 'Le roi malgré lui', 'llm_score': 0.0,...",hotpot_qa,9d3209b2ada754b0578862e54be93ac6,mc,trivia,What was the position of Jean Baptiste Androue...,What was the position of Jean Baptiste Androue...
31740,validation,Who is the lead vocalist for Maroon 5's sixth ...,Adam Levine,"[{'id': 'Overexposed (album)', 'llm_score': 0....",hotpot_qa,ced1dc8407f6d753859c2d70d4a36fe1,mc,trivia,Who is the lead vocalist for Maroon 5's sixth ...,Who is the lead vocalist for Maroon 5's sixth ...


In [46]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on January 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [['The Super Bowl....season since 1966,','replacing the NFL...in February.'], 
    ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 2
})

In [1]:
from pandas import read_parquet

itrf = read_parquet("../data/evaluation_results/itrf_evaluation_llmware_llmware_evaluated.parquet")

In [8]:
itrf[["faithfulness","answer_relevancy", "answer_similarity","context_recall","context_precision"]].mean()

0.22547177478341435

In [63]:
from pandas import DataFrame
result = DataFrame(columns=itrf.columns)

In [87]:
import pandas as pd

result = pd.concat([result, itrf.sample(400), itrf.sample(400)])

len(result)

16800

In [89]:
result.drop_duplicates(inplace=True, subset=["query", "src", "id"])

In [91]:
len(result)

13051